In [1]:
from sec_certs.dataset import CCDataset
from pathlib import Path
import shutil
import subprocess

In [2]:
# A directory with a dump of the documents that will enrich the dataset.
doc_dump_dir = Path("cc_certs_03_10_2022")
print(doc_dump_dir.exists())

# An output directory for the dataset.
dataset_dir = Path("cc_03_10_2022")
print(dataset_dir.exists())

True
True


In [3]:
# Print tool versions
subprocess.run(["pdftotext", "-v"])
subprocess.run(["tesseract", "-v"]);

tesseract 5.2.0
 leptonica-1.82.0
  libgif 5.2.1 : libjpeg 8d (libjpeg-turbo 2.1.3) : libpng 1.6.38 : libtiff 4.4.0 : zlib 1.2.12 : libwebp 1.2.4 : libopenjp2 2.5.0
 Found AVX2
 Found AVX
 Found FMA
 Found SSE4.1
 Found OpenMP 201511
 Found libarchive 3.6.1 zlib/1.2.12 liblzma/5.2.5 bz2lib/1.0.8 liblz4/1.9.3 libzstd/1.5.2
 Found libcurl/7.85.0 OpenSSL/1.1.1q zlib/1.2.12 brotli/1.0.9 zstd/1.5.2 libidn2/2.3.3 libpsl/0.21.1 (+libidn2/2.3.0) libssh2/1.10.0 nghttp2/1.50.0


pdftotext version 22.09.0
Copyright 2005-2022 The Poppler Developers - http://poppler.freedesktop.org
Copyright 1996-2011, 2022 Glyph & Cog, LLC


In [4]:
# Create the dataset
dset = CCDataset(root_dir=dataset_dir, name="cc", description="Final run on 03.10.2022")

In [5]:
# Get the metadata
dset.get_certs_from_web()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.00s/it]
The CSV cc_03_10_2022/web/cc_products_active.csv contains 8 duplicates by the primary key.
The CSV cc_03_10_2022/web/cc_products_archived.csv contains 10 duplicates by the primary key.
When merging certificates with dgst b26ce64e0c677e3d, the following mismatch occured: Attribute=security_level, self[security_level]={'ALC_DVS.1', 'EAL2+'}, other[security_level]={'EAL2'}
When merging certificates with dgst 7f043162f88a1c3a, the following mismatch occured: Attribute=not_valid_after, self[not_valid_after]=2024-09-19, other[not_valid_after]=2024-09-18
When merging certificates with dgst 109eb2158ca6a2f9, the following mismatch occured: Attribute=security_level, self[security_level]={'ADV_IMP.2', 'EAL4+', 'AVA_VLA.4', 'ATE_DPT.2', 'AVA_MSU.3'}, other[security_level]={'ADV_DVS.2', 'AVA_MSU.3', 'EAL5+', 'AVA_VLA.4'}
When merging certifi

In [6]:
# Process the profiles?
dset.process_protection_profiles()

Duplicate entry in PP dataset: ('Protection Profile for Enterprise Security Management Policy Management', 'https://www.commoncriteriaportal.org/files/ppfiles/PP_ESM_PM_V1.4.pdf')
Duplicate entry in PP dataset: ('Stateful Traffic Filter Firewall Extended Package for Network Device Protection Profile', 'https://www.commoncriteriaportal.org/files/ppfiles/PP_ND_TFFWEP_V1.0.pdf')
Duplicate entry in PP dataset: ('Stateful Traffic Filter Firewall Extended Package for Network Device Protection Profile', 'https://www.commoncriteriaportal.org/files/ppfiles/PP_ND_TFFWEP_V1.0.pdf')
Duplicate entry in PP dataset: ('Public Key-Enabled Application Family of Protection Profiles', 'https://www.commoncriteriaportal.org/files/ppfiles/PP_VID3004-PP.pdf')
Duplicate entry in PP dataset: ('Protection Profile for Wireless Local Area Network (WLAN) Access Systems', 'https://www.commoncriteriaportal.org/files/ppfiles/PP_WLAN_AS_V1.0.pdf')
Duplicate entry in PP dataset: ('PC Client Specific Trusted Platform Mod

In [7]:
# Enrich the dataset with the pre-downloaded PDFs (collected over multiple runs to fill in the timeouts and disappearances).
dset.reports_pdf_dir.mkdir(parents=True, exist_ok=True)
dset.targets_pdf_dir.mkdir(parents=True, exist_ok=True)
for cert in dset:
    report_pdf = doc_dump_dir / "report" / "pdf" / f"{cert.dgst}.pdf"
    if report_pdf.exists():
        shutil.copy(report_pdf, cert.state.report_pdf_path)
        cert.state.report_download_ok = True
    target_pdf = doc_dump_dir / "target" / "pdf" / f"{cert.dgst}.pdf"
    if target_pdf.exists():
        shutil.copy(target_pdf, cert.state.st_pdf_path)
        cert.state.st_download_ok = True

In [8]:
# Try to download last missing (or ones not present in the pre-downloaded)
dset.download_all_pdfs(fresh=False)

Cert dgst: 8ac22970bd8042d2 failed to download report from https://www.commoncriteriaportal.org/files/epfiles/st_vid4016b-vr.pdf, code: nok
Cert dgst: 82fcce2db9ef6063 failed to download report from https://www.commoncriteriaportal.org/files/epfiles/emCA%20Certification%20Report%20v1.0.pdf, code: nok
Cert dgst: d79ecebfe21f2144 failed to download report from https://www.commoncriteriaportal.org/files/epfiles/st_vid10024a-vr.pdf, code: nok
Cert dgst: 305af0eec85bb856 failed to download report from https://www.commoncriteriaportal.org/files/epfiles/[KECS-CR-13-28]%20AhnLab%20TrusGuard%20V2.2(??).pdf, code: nok
Cert dgst: a90d78924e91dc6c failed to download report from https://www.commoncriteriaportal.org/files/epfiles/ANSSI-CC_2010-03en1.pdf, code: nok
Cert dgst: 02482228eb547c15 failed to download report from https://www.commoncriteriaportal.org/files/epfiles/st_vid4016a-vr.pdf, code: nok
Cert dgst: 391c8b9254ed6ed9 failed to download report from https://www.commoncriteriaportal.org/fil

In [9]:
# Convert all
dset.convert_all_pdfs()

Converting reports to txt:   2%|██                                                                                         | 115/5112 [00:08<06:00, 13.86it/s]Detected garbage during conversion of cc_03_10_2022/certs/reports/pdf/bfc86f14e2484335.pdf
Error during OCR of cc_03_10_2022/certs/reports/pdf/bfc86f14e2484335.pdf, using garbage: pdftoppm failed: 1
Cert dgst: bfc86f14e2484335 failed to convert report pdf->txt
Converting reports to txt:   4%|███▍                                                                                       | 194/5112 [00:12<05:14, 15.65it/s]Detected garbage during conversion of cc_03_10_2022/certs/reports/pdf/d8c205b4924f91b3.pdf
Detected garbage during conversion of cc_03_10_2022/certs/reports/pdf/20874d4fa6e4c878.pdf
Detected garbage during conversion of cc_03_10_2022/certs/reports/pdf/f8a79eb20ade29af.pdf
Detected garbage during conversion of cc_03_10_2022/certs/reports/pdf/3477723044183b31.pdf
Detected garbage during conversion of cc_03_10_2022/certs/r

Error during OCR of cc_03_10_2022/certs/reports/pdf/f9c8da9deff77ab5.pdf, using garbage: pdftoppm failed: 1
Cert dgst: f9c8da9deff77ab5 failed to convert report pdf->txt
Converting reports to txt:  87%|██████████████████████████████████████████████████████████████████████████████            | 4433/5112 [58:59<00:50, 13.40it/s]Error when converting pdf->txt: poppler error creating document
Detected garbage during conversion of cc_03_10_2022/certs/reports/pdf/7147c2f70d983d57.pdf
Error during OCR of cc_03_10_2022/certs/reports/pdf/7147c2f70d983d57.pdf, using garbage: pdftoppm failed: 1
Cert dgst: 7147c2f70d983d57 failed to convert report pdf->txt
Converting reports to txt:  96%|██████████████████████████████████████████████████████████████████████████████████████    | 4888/5112 [59:34<00:14, 15.80it/s]Detected garbage during conversion of cc_03_10_2022/certs/reports/pdf/1683817e5db17d9c.pdf
Detected garbage during conversion of cc_03_10_2022/certs/reports/pdf/b762fcb2983aa148.pdf
Convert

Error when converting pdf->txt: poppler error creating document
Detected garbage during conversion of cc_03_10_2022/certs/targets/pdf/422dc5758723c7d1.pdf
Error when converting pdf->txt: poppler error creating document
Error when converting pdf->txt: poppler error creating document
Error when converting pdf->txt: poppler error creating document
Detected garbage during conversion of cc_03_10_2022/certs/targets/pdf/4c9468f20fdb04f7.pdf
Detected garbage during conversion of cc_03_10_2022/certs/targets/pdf/a2b962c7f1d2bc56.pdf
Detected garbage during conversion of cc_03_10_2022/certs/targets/pdf/7147c2f70d983d57.pdf
Error during OCR of cc_03_10_2022/certs/targets/pdf/a2b962c7f1d2bc56.pdf, using garbage: pdftoppm failed: 1
Cert dgst: a2b962c7f1d2bc56 failed to convert security target pdf->txt
Error during OCR of cc_03_10_2022/certs/targets/pdf/7147c2f70d983d57.pdf, using garbage: pdftoppm failed: 1
Cert dgst: 7147c2f70d983d57 failed to convert security target pdf->txt
Error during OCR of cc

In [5]:
# Analyze all
dset.analyze_certificates()

Extracting report metadata:   0%|                                                                                                    | 0/5104 [00:00<?, ?it/s]Failed to read metadata of certs/reports/pdf/1e91595cd5e7fece.pdf, error: Unexpected escaped string: b'\x83'
Failed to read metadata of certs/reports/pdf/451e0806dfcfd29d.pdf, error: PDF metadata unavailable
Extracting report metadata:  34%|██████████████████████████████                                                          | 1742/5104 [00:11<00:20, 165.40it/s]Failed to read metadata of certs/reports/pdf/a62c55533f4e1636.pdf, error: PDF metadata unavailable
Failed to read metadata of certs/reports/pdf/d3568613c552f9e8.pdf, error: EOF marker not found
Extracting report metadata:  45%|███████████████████████████████████████▍                                                | 2290/5104 [00:14<00:16, 173.82it/s]Failed to read metadata of certs/reports/pdf/1c5deb472a197eb6.pdf, error: PDF metadata unavailable
Failed to read metadata o

Extracting report metadata:  72%|███████████████████████████████████████████████████████████████                         | 3657/5104 [00:22<00:12, 118.13it/s]Failed to read metadata of certs/reports/pdf/9114f0f9938e558c.pdf, error: PDF metadata unavailable
Failed to read metadata of certs/reports/pdf/08b09322ee9df0af.pdf, error: PDF metadata unavailable
Extracting target metadata:   2%|█▍                                                                                         | 78/5029 [00:01<01:06, 74.44it/s]PdfReadWarning: Multiple definitions in dictionary at byte 0x724b1 for key /Im312 [generic.py:590]
Failed to read metadata of certs/targets/pdf/d47a03be1cb342b1.pdf, error: PDF metadata unavailable
Failed to read metadata of certs/targets/pdf/5af5a1b535422b51.pdf, error: PDF metadata unavailable
Failed to read metadata of certs/targets/pdf/21d6eff0bfe3971f.pdf, error: PDF metadata unavailable
Extracting target metadata:   6%|█████▋                                                   

Failed to read metadata of certs/targets/pdf/bbcfe0aee7ea78ea.pdf, error: PDF metadata unavailable
Failed to read metadata of certs/targets/pdf/f0bd6a29eee94a2c.pdf, error: PDF metadata unavailable
Failed to read metadata of certs/targets/pdf/463ecd64b7506048.pdf, error: PDF metadata unavailable
Extracting report frontpages:   8%|██████▉                                                                                | 404/5104 [00:01<00:12, 384.61it/s]WARNING: multiple rules are matching same certification document: /home/jjan/dev/sec-certs/notebooks/cc_03_10_2022/certs/reports/txt/4844355021ea22f1.txt


parsing cpe matching (by NIST) dictionary: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 397693/397693 [00:26<00:00, 15145.61it/s]
Building-up lookup dictionaries for fast CVE matching: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 196860/196860 [00:09<00:00, 19964.38it/s]
Computing related CVES: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 704/704 [00:00<00:00, 79643.70it/s]


In [6]:
# And do maintenance updates as well
dset.process_maintenance_updates()

ERROR:sec_certs.sample.certificate:Cert dgst: cert_bcac7e3f9f614ec6_update_531b2055de9b34f2 failed to download ST from https://www.commoncriteriaportal.org/files/epfiles/XSMART%20e-Passport%20V1.3_R4_ASE_LITE(ENG)_v1.0.1.pdf, code: 408
ERROR:sec_certs.utils.helpers:Failed to download from https://www.commoncriteriaportal.org/files/epfiles/0915ma3b_pdf.pdf; HTTPSConnectionPool(host='www.commoncriteriaportal.org', port=443): Read timed out.
ERROR:sec_certs.sample.certificate:Cert dgst: cert_73632a66aceeb90e_update_c6e93ff837a633e2 failed to download ST from https://www.commoncriteriaportal.org/files/epfiles/0915ma3b_pdf.pdf, code: nok
ERROR:sec_certs.sample.certificate:Cert dgst: cert_2a91f42389fda90d_update_475e73ebad1af7a2 failed to download ST from https://www.commoncriteriaportal.org/files/epfiles/XSMART%20e-Passport%20V1.4_ASE(BAC%20with%20AA)_ENG_V1.7.pdf, code: 408
ERROR:sec_certs.utils.helpers:Failed to download from https://www.commoncriteriaportal.org/files/epfiles/0945V2ma2b_p

Extracting target keywords: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 448/448 [03:05<00:00,  2.41it/s]


In [7]:
# Finally, dump it all
dset.to_json()